## **Word2Vec with CBOW**

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
import tensorflow as tf
from keras.models import Sequential
import keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Embedding, Lambda, Reshape, Dropout
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## **Read Data**

In [3]:
data = pd.read_csv('data.txt')

In [4]:
data.head()

,1
0,ای رستخیز ناگهان، وی رحمت بی منتها\tای آتشی اف...
1,امروز خندان آمدی، مفتاح زندان آمدی\tبر مستمندا...
2,خورشید را حاجب تویی، امید را واجب تویی\tمطلب ت...
3,در سینه ها برخاسته، اندیشه را آراسته\tهم خویش ...
4,ای روح بخش بی بَدَل، وی لذتِ علم و عمل\tباقی ب...


**Read Stop-Words**

In [5]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [6]:
stopwords = read_stop_words('stopwords.txt')
print(len(stopwords))

1421


**hazm library**

In [7]:
# install hazm library
!pip install hazm
from hazm import word_tokenize

### **Preprocess the text**

In [8]:
# preprocess the text
def text_preprocess(data):
  text = [line.replace('\t', ' ') for line in data.values.flatten()]
  text = [line.replace('-', ' ') for line in text]
  text = [re.sub("\d+", "", t) for t in text]

  word_tokenized = [word_tokenize(t) for t in text]
  word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

  sentences = [' '.join(sentence) for sentence in word_tokenized_filtered]
  sentences = [sentence for sentence in sentences if sentence != '']

  return sentences


In [9]:
sentences = text_preprocess(data)

In [10]:
sentences[0:5]

['رستخیز رحمت منتها آتشی افروخته بیشه اندیشه',
 'خندان آمدی مفتاح زندان آمدی مستمندان آمدی بخشش فضل خدا',
 'خورشید حاجب امید واجب مطلب طالب منتها مبتدا',
 'سینه برخاسته اندیشه آراسته حاجت روا',
 'روح بَدَل لذتِ علم باقی بهانه دغل علت دوا']

## **Tokenizer**

In [11]:
# tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [12]:
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [13]:
list(tokenizer.word_index.items())[0:15]

[('جان', 1),
 ('دل', 2),
 ('عشق', 3),
 ('آب', 4),
 ('چشم', 5),
 ('شب', 6),
 ('جهان', 7),
 ('شمس', 8),
 ('دست', 9),
 ('مست', 10),
 ('گل', 11),
 ('یار', 12),
 ('عقل', 13),
 ('جمله', 14),
 ('غم', 15)]

In [14]:
# find more stopwords by sorting the tokenizer word counts
list(dict(sorted(tokenizer.word_counts.items(), reverse=True, key=lambda t: t[1])).items())[0:20]

[('جان', 748),
 ('دل', 516),
 ('عشق', 415),
 ('آب', 205),
 ('چشم', 183),
 ('شب', 171),
 ('جهان', 170),
 ('شمس', 166),
 ('دست', 163),
 ('مست', 157),
 ('گل', 155),
 ('یار', 133),
 ('عقل', 119),
 ('جمله', 116),
 ('غم', 113),
 ('آتش', 110),
 ('نور', 106),
 ('مه', 105),
 ('خاک', 100),
 ('تن', 100)]

In [15]:
encoded = tokenizer.texts_to_sequences(sentences)

In [16]:
encoded[0:10]

[[2702, 275, 1317, 390, 1592, 946, 135],
 [175, 558, 1102, 391, 558, 4259, 558, 809, 324, 23],
 [46, 2703, 625, 2010, 810, 249, 1317, 4260],
 [88, 4261, 135, 1318, 708, 709],
 [29, 4262, 4263, 212, 153, 392, 1593, 947, 235],
 [1593, 710, 2011, 559, 10, 4264, 10, 200, 2704],
 [4265, 456, 13, 4266, 456, 4267, 200, 4268, 1103, 411],
 [948, 98, 1594, 344, 1104, 1594, 4269, 176, 1594, 4270],
 [4271, 74, 41, 1, 4272, 392, 1595, 1, 250, 4273, 236, 276, 4274, 1105],
 [192, 4275, 711, 56, 212, 2705, 369, 712, 1106, 31, 560, 177]]

In [17]:
num_all_words = sum(len(s) for s in encoded) # total number of words in the corpus
num_unique_words = len(tokenizer.word_index) + 1  # total number of unique words in the corpus

In [18]:
num_all_words, num_unique_words

(38419, 10555)

### **Generate data**

In [19]:
# Parameters
window_size = 2

In [20]:
# CBOW
def generate_data(corpus, window_size, num_unique_words):
    all_inputs = []
    all_outputs = []

    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            start = index - window_size
            end = index + window_size + 1

            context_words = []
            for i in range(start, end):
                if i != index:
                    if 0 <= i < L:
                        context_words.append(sentence[i])
                    else:
                        context_words.append(0)
            all_inputs.append(context_words)
            all_outputs.append(to_categorical(word, num_unique_words))
                 
    return (np.array(all_inputs), np.array(all_outputs))    

In [22]:
# Create training data
X_train, y_train = generate_data(encoded, window_size, num_unique_words)
X_train.shape, y_train.shape

((38419, 4), (38419, 10555))

In [23]:
X_train, y_train

(array([[    0,     0,   275,  1317],
        [    0,  2702,  1317,   390],
        [ 2702,   275,   390,  1592],
        ...,
        [   22,    24,   609,  1793],
        [   24, 10554,  1793,     0],
        [10554,   609,     0,     0]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

## **Create Neural Network**

In [24]:
embed_size=50
model = Sequential()
model.add(Embedding(input_dim=num_unique_words,
                    output_dim=embed_size,
                    input_length=window_size*2,
                    name="embedding_layer",
                    embeddings_initializer='glorot_uniform'))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size, )))
model.add(Dense(num_unique_words, activation='softmax', kernel_initializer='glorot_uniform'))

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 4, 50)            527750    
                                                                 
 lambda (Lambda)             (None, 50)                0         
                                                                 
 dense (Dense)               (None, 10555)             538305    
                                                                 
Total params: 1,066,055
Trainable params: 1,066,055
Non-trainable params: 0
_________________________________________________________________


In [27]:
history = model.fit(X_train, y_train, epochs=300, verbose=1, batch_size=128)

Epoch 1/300
301/301 [==============================] - 13s 41ms/step - loss: 8.9789 - accuracy: 0.0188
Epoch 2/300
301/301 [==============================] - 12s 41ms/step - loss: 8.3275 - accuracy: 0.0195
Epoch 3/300
301/301 [==============================] - 12s 41ms/step - loss: 8.1973 - accuracy: 0.0196
Epoch 4/300
301/301 [==============================] - 13s 42ms/step - loss: 8.1371 - accuracy: 0.0197
Epoch 5/300
301/301 [==============================] - 12s 41ms/step - loss: 8.0929 - accuracy: 0.0199
Epoch 6/300
301/301 [==============================] - 12s 39ms/step - loss: 8.0540 - accuracy: 0.0202
Epoch 7/300
301/301 [==============================] - 13s 42ms/step - loss: 8.0140 - accuracy: 0.0203
Epoch 8/300
301/301 [==============================] - 12s 39ms/step - loss: 7.9690 - accuracy: 0.0207
Epoch 9/300
301/301 [==============================] - 12s 40ms/step - loss: 7.9166 - accuracy: 0.0220
Epoch 10/300
301/301 [==============================] - 12s 40ms/step - l

### **Save Model**

In [28]:
model.save('model_cbow.h5')

**Load Model**

In [29]:
# laod model
model = load_model('model_cbow.h5')

In [30]:
model

**Get embedding weights**

In [31]:
embedding_weights = model.get_layer('embedding_layer').get_weights()[0]

In [32]:
len(embedding_weights)

10555

In [33]:
embedding_weights[2]

array([-0.37890863,  1.2352958 , -0.6524299 , -0.18644147, -0.09871053,
       -1.0932955 , -0.12363061,  0.10171067,  0.55119735,  0.37862375,
       -0.5103982 ,  0.20892195,  0.29294518,  0.41166008, -0.3425361 ,
        0.18025835, -0.52293026,  0.47391433, -0.08354196, -0.13468057,
       -0.8889005 ,  0.11866476, -0.16066231, -0.66750354, -0.49301612,
       -0.49759975, -0.8288309 ,  0.07370132, -0.335891  ,  0.16096488,
       -0.2431528 ,  0.34740824,  0.24899083, -0.20487566, -1.3941584 ,
       -0.14084804,  0.39803383,  0.6777012 , -0.13455176,  0.50267935,
       -0.6485454 ,  0.310815  ,  0.16380094, -0.36356407, -0.00425572,
       -0.08583095,  0.18422739,  0.4395408 ,  0.42269558, -0.7542688 ],
      dtype=float32)

In [34]:
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [35]:
from sklearn.metrics.pairwise import euclidean_distances

def get_most_similarity(word, weights= embedding_weights, tokenizer=tokenizer, n=20):
  distance_matrix = euclidean_distances(weights)
  word_to_sequences = tokenizer.texts_to_sequences([word])[0][0]
  index = np.argsort(distance_matrix[word_to_sequences])[:n]
  sequences_to_word = tokenizer.sequences_to_texts([index])[0]
  most_similarity = sequences_to_word.split(' ')
  return most_similarity

In [36]:
get_most_similarity('گل')

['گل',
 'خار',
 'زمین',
 'رود',
 'آتش',
 'پا',
 'جو',
 'آسمان',
 'یار',
 'روان',
 'جوی',
 'حیات',
 'چرخ',
 'موسی',
 'درد',
 'رنگ',
 'بحر',
 'خانه',
 'شکر',
 'ریحان']

In [37]:
get_most_similarity('مست')

['مست',
 'آمدست',
 'خواجه',
 'کف',
 'جام',
 'شراب',
 'ره',
 'خراب',
 'گشت',
 'پرده',
 'پاره',
 'خدا',
 'خواب',
 'خاک',
 'نگر',
 'مخمور',
 'نعره',
 'پنهان',
 'خمار',
 'خندان']

In [38]:
get_most_similarity('دست')

['دست',
 'پا',
 'کف',
 'پای',
 'خون',
 'خاک',
 'خورشید',
 'عقل',
 'رخ',
 'جام',
 'رنگ',
 'سجده',
 'پرده',
 'خانه',
 'ماه',
 'خار',
 'گشت',
 'موسی',
 'چرخ',
 'باده']

In [39]:
get_most_similarity('شمس')

['شمس',
 'تبریز',
 'تبریزی',
 'الدین',
 'الحق',
 'دین',
 'مه',
 'نور',
 'ذره',
 'سجده',
 'رخ',
 'روح',
 'دست',
 'خداوند',
 'خورشید',
 'خاک',
 'شاه',
 'عالم',
 'صلاح',
 'ماه']

In [40]:
get_most_similarity('تبریزی')

['تبریزی',
 'تبریز',
 'دین',
 'الدین',
 'الحق',
 'مه',
 'ذره',
 'سجده',
 'خداوند',
 'خورشید',
 'رخ',
 'نور',
 'شاه',
 'عالم',
 'خاک',
 'جمال',
 'آفتاب',
 'روح',
 'خدمت',
 'دست']

In [41]:
get_most_similarity('عقل')

['عقل',
 'درد',
 'خون',
 'باده',
 'فنا',
 'گشته',
 'خاک',
 'عاشقان',
 'زمین',
 'جام',
 'آتش',
 'کش',
 'خم',
 'باغ',
 'آسمان',
 'نقش',
 'نظر',
 'پای',
 'خواب',
 'مرده']

In [42]:
get_most_similarity('غم')

['غم',
 'باغ',
 'گرد',
 'پاک',
 'نقش',
 'گشته',
 'رود',
 'بستان',
 'سخن',
 'آرد',
 'زنده',
 'حیات',
 'هزاران',
 'عاشقان',
 'تنگ',
 'گشت',
 'نهان',
 'نهد',
 'خبر',
 'ره']

In [43]:
get_most_similarity('انگور')

['انگور',
 'روانه',
 'افغان',
 'کران',
 'سیما',
 'فزای',
 'قیر',
 'همو',
 'احسنت',
 'نخواهد',
 'شوره',
 'شیران',
 'غلغله',
 'رخی',
 'دردی',
 'دانا',
 'حسنش',
 'یکتا',
 'اندیش',
 'ریز']

In [44]:
get_most_similarity('آب')

['آب',
 'جوی',
 'جو',
 'روان',
 'سنگ',
 'حیات',
 'آتش',
 'چشمه',
 'خون',
 'گل',
 'زمین',
 'خار',
 'رود',
 'باغ',
 'جهان',
 'بحر',
 'آسمان',
 'عقل',
 'درد',
 'دریا']

In [45]:
get_most_similarity('آتش')

['آتش',
 'خون',
 'درد',
 'عقل',
 'زمین',
 'گشته',
 'خار',
 'رود',
 'آسمان',
 'فنا',
 'جو',
 'باغ',
 'رنگ',
 'چرخ',
 'موسی',
 'عاشقان',
 'حیات',
 'نقش',
 'سو',
 'باده']

In [46]:
get_most_similarity('چشم')

['چشم',
 'خواب',
 'سو',
 'خون',
 'باده',
 'گوش',
 'ساقی',
 'ای',
 'آتش',
 'پنهان',
 'عقل',
 'مست',
 'آسمان',
 'خاک',
 'درد',
 'فنا',
 'رخ',
 'جام',
 'عاشقان',
 'خواهی']

In [47]:
get_most_similarity('شب')

['شب',
 'ماه',
 'عالم',
 'رخ',
 'آفتاب',
 'شمع',
 'پرده',
 'خورشید',
 'عاشقان',
 'شاه',
 'فلک',
 'دوش',
 'لقا',
 'عاشق',
 'خواب',
 'باده',
 'سجده',
 'دور',
 'موسی',
 'نقش']

In [48]:
get_most_similarity('جهان')

['جهان',
 'تن',
 'باغ',
 'درد',
 'پاک',
 'باده',
 'گشته',
 'عاشقان',
 'لطف',
 'گرد',
 'فنا',
 'کش',
 'غم',
 'نهان',
 'خون',
 'حیات',
 'نقش',
 'عقل',
 'خم',
 'جو']

In [49]:
get_most_similarity('دل')

['دل',
 'خون',
 'غم',
 'تن',
 'عقل',
 'آتش',
 'جهان',
 'فنا',
 'پای',
 'خانه',
 'گرد',
 'زمین',
 'سو',
 'باغ',
 'آسمان',
 'رود',
 'عاشقان',
 'کش',
 'خم',
 'درد']

In [50]:
get_most_similarity('مه')

['مه',
 'رخ',
 'ماه',
 'خورشید',
 'سجده',
 'آفتاب',
 'فلک',
 'عالم',
 'لقا',
 'شاه',
 'خداوند',
 'شمع',
 'جمال',
 'پرده',
 'شب',
 'روح',
 'نما',
 'دوش',
 'رنگ',
 'چرخ']

In [51]:
get_most_similarity('نور')

['نور',
 'مه',
 'رخ',
 'اصل',
 'خداوند',
 'فلک',
 'خورشید',
 'سجده',
 'شمع',
 'جمال',
 'لقا',
 'نقاب',
 'پرده',
 'دولت',
 'عالم',
 'جمع',
 'صدر',
 'خدمت',
 'دلا',
 'آفتاب']

In [52]:
get_most_similarity('خاک')

['خاک',
 'جام',
 'باده',
 'کش',
 'زمین',
 'باقی',
 'درد',
 'سجده',
 'لطف',
 'چرخ',
 'بنده',
 'پاک',
 'گشته',
 'رنگ',
 'پاره',
 'عقل',
 'خون',
 'جو',
 'نقش',
 'قدح']

In [53]:
get_most_similarity('تن')

['تن',
 'جهان',
 'باغ',
 'پاک',
 'غم',
 'فزا',
 'عاشقان',
 'گشته',
 'گرد',
 'لطف',
 'حیات',
 'خون',
 'سخن',
 'جو',
 'باده',
 'مرده',
 'پاره',
 'نقش',
 'نهان',
 'درد']

In [55]:
get_most_similarity('کنج')

['خون',
 'عقل',
 'فنا',
 'درد',
 'خاک',
 'باده',
 'زمین',
 'آتش',
 'گشته',
 'عاشقان',
 'آسمان',
 'کش',
 'باغ',
 'جو',
 'سو',
 'خم',
 'خواب',
 'پای',
 'جام',
 'لطف']